## Import Libraries

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import seaborn as sns

## Dataset Loading



In [2]:
# Upload file to Local Drive :

from google.colab import files
uploaded = files.upload()

Saving uber_rides_data.xlsx - sample_train.csv to uber_rides_data.xlsx - sample_train.csv


In [76]:
import io
df = pd.read_csv(io.BytesIO(uploaded['uber_rides_data.xlsx - sample_train.csv']))  # Dataset is now stored in a Pandas Dataframe


In [77]:
display(df.head(5))

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [78]:
df.shape    # To calculate shape of given dataset

(200000, 8)

## Data Preprocessing

In [79]:
# To know the details of the DataFrame and datatype of every column :

df.info()                     # raw data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


From above information obtained from the Uber rides dataframe, it can be observed that :
* There are total 200000 rows and 8 columns.
* Moreover, Null values are present in "dropoff_longitude", "dropoff_latitude" columns.

In [80]:
# To check null/missing values count (if present) in each column :

df.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [81]:
# Checking for duplicate records :

df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
199995    False
199996    False
199997    False
199998    False
199999    False
Length: 200000, dtype: bool

<b>Duplicated() :</b> By default, for each set of duplicated values, the first occurrence is set on False and all others on True.

In [82]:
# Count of Duplicate Records in the Entire Dataset :

df.duplicated().sum()

0

A sum operation on the results obtained from the duplicated() function will give us the total number of duplicate values.
- As displayed above, our dataframe has no Duplicates.

## Task-1 :
* Convert 'pickup_datetime' to datetime datatype
* Remove missing values
* Calculate Average fare amount




In [83]:
# Another way to convert 'pickup_datetime' to datetime datatype

df['pickup_datetime'].astype('datetime64[ns]')

0        2015-05-07 19:52:06
1        2009-07-17 20:04:56
2        2009-08-24 21:45:00
3        2009-06-26 08:22:21
4        2014-08-28 17:47:00
                 ...        
199995   2012-10-28 10:49:00
199996   2014-03-14 01:09:00
199997   2009-06-29 00:42:00
199998   2015-05-20 14:56:25
199999   2010-05-15 04:08:00
Name: pickup_datetime, Length: 200000, dtype: datetime64[ns]

In [84]:
# Converting 'pickup_datetime' to datetime datatype

df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [85]:
# Removing null values

df.dropna(inplace=True)

In [86]:
df.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [87]:
df.info()    # Cleaned data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199999 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   ride_id            199999 non-null  int64              
 1   fare_amount        199999 non-null  float64            
 2   pickup_datetime    199999 non-null  datetime64[ns, UTC]
 3   pickup_longitude   199999 non-null  float64            
 4   pickup_latitude    199999 non-null  float64            
 5   dropoff_longitude  199999 non-null  float64            
 6   dropoff_latitude   199999 non-null  float64            
 7   passenger_count    199999 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(2)
memory usage: 13.7 MB


In [88]:
# Calculating Average fare amount

df["fare_amount"].mean()

11.359891549457748

## Task-2 : Calculating Haversine distance

In [89]:
import math

# Function to calculate Haversine distance between each pickup and dropoff

def haversine_dist(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    radius = 6371

    # Haversine formula
    dlat = math.radians(lat2-lat1)     # Converting latitude and longitude to radians
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c    # distance

    return d

# Haversine distance for each record
df['Haversine_d'] = df.apply(lambda row: haversine_dist(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

# Median Haversine Distance between pickup and dropoff location
median_dist = df['Haversine_d'].median()

print("Median Haversine Distance between pickup and dropoff location in kilometers : {} ".format(median_dist))
print("Median Haversine Distance between pickup and dropoff location in miles : {} ".format((median_dist)*0.621371))

Median Haversine Distance between pickup and dropoff location in kilometers : 2.1209923961838655 
Median Haversine Distance between pickup and dropoff location in miles : 1.3179231662091646 


In [90]:
# Maximum Haversine Distance between pickup and dropoff location

Max_Haversine_d = df['Haversine_d'].max()
print("Median Haversine Distance between pickup and dropoff location in kilometers : {} ".format(Max_Haversine_d))
print("Median Haversine Distance between pickup and dropoff location in miles : {} ".format((Max_Haversine_d)*0.621371))

Median Haversine Distance between pickup and dropoff location in kilometers : 16409.23913531318 
Median Haversine Distance between pickup and dropoff location in miles : 10196.225330748686 


In [91]:
# No.of rides that have 0.0 haversine distance between pickup and dropoff location :

rides = (df['Haversine_d'] == 0.0).sum()
print("No.of rides that have 0.0 haversine distance between pickup and dropoff location :", rides)

No.of rides that have 0.0 haversine distance between pickup and dropoff location : 5632


In [92]:
# Mean 'fare_amount' for rides with 0 haversine distance

avg_fare_zero_d = df[df['Haversine_d'] == 0.0]['fare_amount'].mean()
print(" Average 'fare_amount' for rides with 0 haversine distance :", avg_fare_zero_d)

 Average 'fare_amount' for rides with 0 haversine distance : 11.585317826704546


In [93]:
# Maximum 'fare_amount' for a ride

max_fare = df['fare_amount'].max()
print("Maximum 'fare_amount' for a ride:", max_fare)

Maximum 'fare_amount' for a ride: 499.0


In [94]:
# Haversine distance between pickup and dropoff location for the costliest ride

# the highest 'fare_amount
costliest_ride = df[df['fare_amount'] == df['fare_amount'].max()]

# haversine distance for the costliest ride
costliest_ride_d = costliest_ride['Haversine_d'].values[0]

print("Haversine distance between pickup and dropoff location for the costliest ride in kilometers: ", costliest_ride_d)
print("Haversine distance between pickup and dropoff location for the costliest ride in miles: ", costliest_ride_d*0.621371)

Haversine distance between pickup and dropoff location for the costliest ride in kilometers:  0.0007899213197627204
Haversine distance between pickup and dropoff location for the costliest ride in miles:  0.0004908342003822813


In [95]:
# No.of rides were recorded in the year 2014

df['pickup_year'] = df['pickup_datetime'].dt.year
rides_2014 = len(df[df['pickup_year'] == 2014])

print("No.of rides recorded in the year 2014:", rides_2014)

No.of rides recorded in the year 2014: 29968


In [96]:
# No.of rides were recorded in the first quarter of 2014

df['pickup_year'] = df['pickup_datetime'].dt.year
df['pickup_quarter'] = df['pickup_datetime'].dt.quarter
rides_1quart_2014 = len(df[(df['pickup_year'] == 2014) & (df['pickup_quarter'] == 1)])

print("No.of rides were recorded in the first quarter of 2014:", rides_1quart_2014)


No.of rides were recorded in the first quarter of 2014: 7687


In [97]:
# Day of the week in September 2010, in which maximum rides were recorded

df['ride_week_day'] = df['pickup_datetime'].dt.day_name()

sept_rides_2010 = df[(df['pickup_datetime'].dt.year == 2010) & (df['pickup_datetime'].dt.month == 9)]

max_rides = sept_rides_2010['ride_week_day'].mode()[0]

print("Day of the week in September 2010, in which maximum rides were recorded:", max_rides)

Day of the week in September 2010, in which maximum rides were recorded: Thursday


## Task-3 :
Apply a Machine Learning Algorithm to predict the fare amount given following input features:
passenger_count, distance and ride_week_day.

Perform a 70-30 split of data.

In [98]:
df.head(5)

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Haversine_d,pickup_year,pickup_quarter,ride_week_day
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,2015,2,Thursday
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,2009,3,Friday
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,2009,3,Monday
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,2009,2,Friday
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,2014,3,Thursday


### Splitting the data

In [100]:
Y = df['fare_amount']
df1 = df.drop(columns = 'fare_amount')

In [161]:
# One-hot encoding for 'ride_week_day' column

df1 = pd.get_dummies(df1, columns=['ride_week_day'], prefix='day')

In [162]:
X = df1[['passenger_count', 'Haversine_d', 'day_Monday', 'day_Tuesday', 'day_Wednesday', 'day_Thursday', 'day_Friday']]

In [163]:
X

,passenger_count,Haversine_d,day_Monday,day_Tuesday,day_Wednesday,day_Thursday,day_Friday
0,1,1.683323,0,0,0,1,0
1,1,2.457590,0,0,0,0,1
2,1,5.036377,1,0,0,0,0
3,3,1.661683,0,0,0,0,1
4,5,4.475450,0,0,0,1,0
...,...,...,...,...,...,...,...
199995,1,0.112210,0,0,0,0,0
199996,1,1.875050,0,0,0,0,1
199997,2,12.850319,1,0,0,0,0
199998,1,3.539715,0,0,1,0,0


In [164]:
Y

0          7.5
1          7.7
2         12.9
3          5.3
4         16.0
          ... 
199995     3.0
199996     7.5
199997    30.9
199998    14.5
199999    14.1
Name: fare_amount, Length: 199999, dtype: float64

### Split the data into training data and test data using sklearn's train-test-split

In [166]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

In [168]:
X_conti = X[['Haversine_d']]
X_catg = X[['passenger_count','day_Monday', 'day_Tuesday', 'day_Wednesday', 'day_Thursday', 'day_Friday']]

In [169]:
X_Train,X_Test,Y_Train,Y_Test = train_test_split(X, Y,
                                                 test_size = 0.30,
                                                 random_state = 20)

In [170]:
display(X_Train.shape,X_Test.shape,len(Y_Train),len(Y_Test))


(139999, 7)

(60000, 7)

139999

60000

In [187]:
ML_algorithms = {'Linear Regression': LinearRegression(),
    'Decision Tree Classifier': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'KNN Classifier': KNeighborsRegressor()}

adjusted_r2_scores = {}

for algo_name, algo in ML_algorithms.items():
    algo.fit(X_Train, Y_Train)
    Y_pred = algo.predict(X_Test)

    r2 = r2_score(Y_Test, Y_pred)     # Calculating R-squared value

    n = len(X_Test)
    p = X_Test.shape[1]
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))      # Calculating adjusted R-squared value

    adjusted_r2_scores[algo_name] = adjusted_r2

    print(f"{algo_name}", "algorithm :")
    print(f"R-squared value: {r2:.2f}")
    print(f"Adjusted R-squared value: {adjusted_r2:.2f}",end="\n\n")


Linear Regression algorithm :
R-squared value: 0.00
Adjusted R-squared value: 0.00

Decision Tree Classifier algorithm :
R-squared value: 0.52
Adjusted R-squared value: 0.52

Random Forest Regression algorithm :
R-squared value: 0.69
Adjusted R-squared value: 0.69

KNN Classifier algorithm :
R-squared value: 0.68
Adjusted R-squared value: 0.68



In [188]:
# ML algorithm with the least adjusted R-squared value :
least_adjusted_r2_algo = min(adjusted_r2_scores, key=adjusted_r2_scores.get)

print("ML algorithm with the least adjusted R-squared value is :", least_adjusted_r2_algo)


ML algorithm with the least adjusted R-squared value is : Linear Regression
